In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [19]:
import cv2
import numpy as np
import pandas as pd
import time
try:
    from PIL import Image
except ImportError:
    import Image
# import pytesseract
import os

In [20]:
def read_img(path):
  img = cv2.imread(path,0)
  return img

In [21]:
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [22]:
## To Detect biggest outer box
def imgtocontours(img):
    #thresholding the image to a binary image
    thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    #inverting the image 
    img_bin = 255-img_bin

    # countcol(width) of kernel as 100th of total width
    kernel_len = np.array(img).shape[1]//100
    # Defining a vertical kernel to detect all vertical lines of image 
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    # Defining a horizontal kernel to detect all horizontal lines of image
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    #Use vertical kernel to detect and save the vertical lines in a jpg
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    #Use horizontal kernel to detect and save the horizontal lines in a jpg
    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Combine horizontal and vertical lines in a new third image, with both having same weight.
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
    #Eroding and thesholding the image
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    bitxor = cv2.bitwise_xor(img,img_vh)
    bitnot = cv2.bitwise_not(bitxor)

    # Detect contours for following box detection
    contours, hierarchy = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort all the contours by top to bottom.
    contours, boundingBoxes = sort_contours(contours, method="top-to-bottom")
    #Creating a list of heights for all detected boxes
    heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]
    #Get mean of heights
    mean = np.mean(heights)
    
    return contours,mean,bitnot

In [23]:
def store_boxes(contours,img):
  #Create list bigBox to store all boxes in  
  bigBox = []
  # Get position (x,y), width and height for every contour and show the contour on image
  for c in contours:
      x, y, w, h = cv2.boundingRect(c)
      if (w>200 and h>200):
          bigBox.append([x,y,w,h])

  bigBox = bigBox[-1]
  providedCropedImage = False
  if (img.shape[0] - bigBox[-1]) < 10 :
      providedCropedImage = True 
      
  return providedCropedImage,bigBox

In [24]:
## detect boxes and get size of ractangle which we have to create
def crop_image(bigBox,img):
  x,y,xe,ye = bigBox
  xe = xe+x
  ye = ye+y
  h = img.shape[0]
  w = img.shape[1]
  img = img[y+27+2:ye,x+123:xe-2]
  return img

In [25]:
def store_blackbox(contours):
  box = []
  flag = False
  countCols = 2
  firstcolcheck=True
  blackBox = [] # will get all the dimensions of continuous blackbox

  # Get position (x,y), width and height for every contour and show the contour on image
  for c in contours:
      x, y, w, h = cv2.boundingRect(c)
      if (w<1000 and h<500):
          box.append([x,y,w,h])

          cellWidth = box[-1][2]
          cellHeight= box[-1][3]

          if len(box)>2:
              px = box[-2][0]
              if firstcolcheck :
                  if box[-2][2] == box[-1][2]:
                      countCols += 1
                  else:
                      countCols = (3*countCols +1)
                      COUNTCOLS = countCols
                      firstcolcheck = False
                      
              if not firstcolcheck:
                  countCols -= 1 

              xdiff = px - x
              if (x+box[-2][2]-5)<px<(x+box[2][2]+5):
                  pass
              elif (x+w) > 10 & xdiff != 104 and xdiff>10 and x>10 and px >10 and xdiff>10:
                  if countCols < 0 and box[-1][2] == (box[-2][2]+3):
                      leftPoint = (x+w)
                      rightPoint = px - 4 - 3 
                      distanceFromTop = y - 1
                      cellW = box[-2][2] + 3 +3

                  else:
                      leftPoint = (x+w)
                      rightPoint = px - 4
                      distanceFromTop = y - 1
                      cellW = box[-2][2] + 3

                  cellH = box[-2][3] + 3
                  xs = leftPoint
                  rxe = rightPoint
                  ys = distanceFromTop
                  blackBox.append((xs,rxe,ys,cellW,cellH))
              if countCols < 0 and box[-1][2]>75:
                  break
  return blackBox

In [26]:
def drawrectangles(img,xs,ys,xe,ye,rxe,rye,rows,columns,cellW,cellH,providedCropedImage):

    img = cv2.rectangle(img, (xs+1,ys-1), (rxe,rye), (255,255,255),-1 ) # to create big white box in place of black box
    xs = xs
    ys = ys - 2 
    ye = ye - 2 # to create perfect square
    flag = False
    orgys = ys

    if providedCropedImage:
        for i in range(rows):
            nxs = xs + (i)*cellW
            nxe = xe + i*cellW
            ys = orgys
            for j in range(columns):
                nys = ys + j*cellH
                img = cv2.rectangle(img, (nxs,nys), (nxe,ye), (0,0,0), 1)
    else:

        for i in range(rows):
            nxs = xs + (i)*cellW
            nxe = xe + i*cellW
            ys = orgys
            if flag:
                ys = orgys
            for j in range(columns):
                if j%5 == 0 and j!=0:
                    ys = ys-1
                    flag = True
                nys = ys + j*cellH
                img = cv2.rectangle(img, (nxs,nys), (nxe,ye), (0,0,0), 1)
    return img

In [27]:
def whitebox(blackBox,img,providedCropedImage):
  for box in blackBox:
    xs,rxe,ys,cellW,cellH = box
    h,w = img.shape
    rye = h - 4
    xe = xs+(cellW)
    ye = ys+(cellH)
    rows = (rxe-xs)//(cellW-1)
    columns = (rye-ys)//(cellH-1)

    img = drawrectangles(img,xs,ys,xe,ye,rxe,rye,rows,columns,cellW,cellH,providedCropedImage)
  return img

In [28]:
def store(contours,img,mean):
  box = []
  # Get position (x,y), width and height for every contour and show the contour on image
  for c in contours:
      x, y, w, h = cv2.boundingRect(c)
      if (w<1000 and h<500):
          image = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
          box.append([x,y,w,h])

  # Create lists to store cell location 
  row=[]
  column=[]
  j=0

  # Sorting the boxes to their respective row and column
  for i in range(len(box)):    
          
      if(i==0):
          column.append(box[i])
          previous=box[i]    
      else:
          if(box[i][1]<=previous[1]+mean/2):
              column.append(box[i])
              previous=box[i]            
              
              if(i==len(box)-1):
                  row.append(column)        
          else:
              row.append(column)
              column=[]
              previous = box[i]
              column.append(box[i])

  # calculating maximum number of cells
  countcol = 0
  for i in range(len(row)):
      countcol = len(row[i])
      if countcol > countcol:
          countcol = countcol

  #Retrieving the center of each column
  center = [int(row[i][j][0]+row[i][j][2]/2) for j in range(len(row[i])) if row[0]]

  center=np.array(center)
  center.sort()

  # Regarding the distance to the columns center, the boxes are arranged in respective order

  finalboxes = []
  for i in range(len(row)):
      lis=[]
      for k in range(countcol):
          lis.append([])
      for j in range(len(row[i])):
          diff = abs(center-(row[i][j][0]+row[i][j][2]/4))
          minimum = min(diff)
          indexing = list(diff).index(minimum)
          lis[indexing].append(row[i][j])
      finalboxes.append(lis)

  return finalboxes,row,countcol


In [29]:
def extract_text(finalboxes,bitnot):
  outer=[]
  for i in range(len(finalboxes)):
      for j in range(len(finalboxes[i])):
          inner=''
          if(len(finalboxes[i][j])==0):
              outer.append(' ')
          else:
              for k in range(len(finalboxes[i][j])):
                  y,x,w,h = finalboxes[i][j][k][0],finalboxes[i][j][k][1], finalboxes[i][j][k][2],finalboxes[i][j][k][3]
                  finalimg = bitnot[x:x+h, y:y+w]
                  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1))
                  border = cv2.copyMakeBorder(finalimg,2,2,2,2, cv2.BORDER_CONSTANT,value=[255,255])
                  resizing = cv2.resize(border, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                  dilation = cv2.dilate(resizing, kernel,iterations=1)
                  erosion = cv2.erode(dilation, kernel,iterations=2)
                  # im_pil = Image.fromarray(erosion)
                  # out = tesserocr.image_to_text(im_pil)
                  # print(out)
                  
                  out = pytesseract.image_to_string(erosion,config='-l eng --oem 3 --psm 10 --dpi 300')
                  if(len(out)==0):
                      out = ''
                      # out = pytesseract.image_to_string(erosion,config='-l eng --oem 3 --psm 10 --dpi 300')
                  inner = inner +" "+ out
              outer.append(inner)

  for i in range(len(outer)):
      if outer[i] == ' ist\n\x0c':
          outer[i] = ' 1st\n\x0c'
      if '40' in outer[i]:
        outer[i] = ' 20\"/40\"\n\x0c'
      if outer[i] == ' xX\n\x0c':
          outer[i] = ' X\n\x0c'
      if outer[i] == ' Oo\n\x0c':
          outer[i] = ' \n\x0c'
  return outer

In [30]:
def generate_csv(outer,row,countcol,csv_path):
  arr = np.array(outer)
  df = pd.DataFrame(arr.reshape(len(row), countcol))
  df.to_csv(csv_path)

In [31]:
def main(path):
  img = read_img(path)
  contours,mean,bitnot = imgtocontours(img)
  providedCropedImage,bigBox = store_boxes(contours,img)
  img = crop_image(bigBox,img)
  contours,mean,bitnot = imgtocontours(img)
  blackBox = store_blackbox(contours)
  img = whitebox(blackBox,img,providedCropedImage)
  contours,mean,bitnot = imgtocontours(img)
  finalboxes,row,countcol = store(contours,img,mean)
  outer = extract_text(finalboxes,bitnot)
  print('ocr done.')
  csv_path = os.path.splitext(os.path.basename(path))[0] + '.csv'
  generate_csv(outer,row,countcol,csv_path)

In [ ]:
path = '/content/2.png'
start = time.time()
main(path)
end = time.time()
print(end-start)

In [ ]:
# !apt-get install libleptonica-dev libtesseract-dev

In [ ]:
# pip install tesserocr

In [16]:
# from tesserocr import get_languages

# print(get_languages('/usr/share/tesseract-ocr/4.00/tessdata/'))

('/usr/share/tesseract-ocr/4.00/tessdata/', ['eng'])


In [ ]:
# ! wget https://github.com/tesseract-ocr/tessdata_fast/blob/main/eng.traineddata
# ! sudo mv "/content/eng.traineddata" "/usr/share/tesseract-ocr/4.00/tessdata"

In [17]:
import tesserocr
from PIL import Image

image = Image.open('2.png')
print(tesserocr.image_to_text(image))

 

TTI EMPTY RECEIVING - Subject to Change

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

famsi2022 | ‘2i1e/2022 | 12/19/2022 | 12/20/2022 | 42/24/2022 | 12/22/2022
Ocean Carrier | SIZETYPE |_ Thursday Friday Monday Tuesday | Wednesday | Thursday
fst | 2nd | ast [ 2nd [ist] 2nd | ist | 2nd | tst [2nd | ist_[ 2nd
20g [UAL | SINGLE DUAL | DUAL [SINGLE [DUAL SINGLE
sc Reefer x [Xx x |x Dx] x x
Specie [| x | x x Lx Lx Lx x
204g [SINGLE [SINGLE SINGLE SINGLE [SINGLE [SINGLE SINGLE
* 4% [SNGLE[SINGLE SINGLE [SINGLE [SINGLE | SINGLE SINGLE
Reefer _ [SINGLE | SINGLE x x Lx Lx x
MAERSK | Speciatyeo | x x x x x x x
204g [SINGLE [SINGLE DUAL | DUAD [DUAL | SINGLE SINGLE
4% [SNGLE[SINGLE x | x [x Px x
Specie [ x |X x Lx Lx Lx x
204g [SINGLE [SINGLE x >x fx fx x
& x [x x Lx >x x x
: Reefer x x x x x x x
Specie [| x | x x Lx Lx Lx x
204g [SINGLE [SINGLE